In [2]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 55.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import os
import pandas as pd
from bert_score import score
import spacy
import zipfile

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Paths
original_sop_file = "dataset.xlsx"
generated_zip_file = "sop_generated_tinyllama.zip"  # Path to your zip file

# Original longform file
df = pd.read_excel(original_sop_file)
original_sops = df["cleaned_text"].tolist()

# Evaluating each generated SOP from the zip file
with zipfile.ZipFile(generated_zip_file, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        if file_info.filename.endswith(".txt"):
            with zip_ref.open(file_info.filename) as f:
                content = f.read().decode('utf-8').strip() # Decode content if necessary

            # Removing metadata like "Word count" or headers
            lines = content.splitlines()
            clean_lines = [line for line in lines if not line.lower().startswith("word count") and "===" not in line]
            generated_text = " ".join(clean_lines).strip()


            generated_list = [generated_text] * len(original_sops)

            # BERTScore
            P, R, F1 = score(original_sops, generated_list, lang="en", verbose=False)
            avg_f1 = F1.mean().item()

            # spaCy cosine similarity
            generated_doc = nlp(generated_text)
            similarities = [generated_doc.similarity(nlp(orig)) for orig in original_sops]
            avg_spacy = sum(similarities) / len(similarities)

            print(f"\n File: {file_info.filename}") # Print filename from zip
            print(f"→ Avg BERTScore F1: {avg_f1:.4f}")
            print(f"→ Avg spaCy Similarity: {avg_spacy:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...


 File: sop_datascience_20250411_172449.txt
→ Avg BERTScore F1: 0.8151
→ Avg spaCy Similarity: 0.9886


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 File: sop_datascience_20250411_183451.txt
→ Avg BERTScore F1: 0.8158
→ Avg spaCy Similarity: 0.9818


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 File: sop_Robotics_20250411_173110.txt
→ Avg BERTScore F1: 0.8137
→ Avg spaCy Similarity: 0.9855
